In [1]:
# 必要なライブラリのインポート
import numpy as np
import tensorflow as tf

In [2]:
# ANDゲートの入力値　0と1の組み合わせの行列(4, 2)
X = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]])
# ANDゲートの出力　正解ラベルの行列(4, 1)
T = np.array([[0],
              [0],
              [0],
              [1]])

In [3]:
learning_rate=0.1   # 学習率
training_epochs=100 # 学習回数

In [4]:
'''
データフローグラフ
'''
#---- 変数 ----#
# 重み(2,1)の行列(縦ベクトル)
# 切断正規分布の母集団からランダムサンプリング
w = tf.Variable(tf.truncated_normal([2, 1]))
# バイアスは0で初期化（形状は要素数1の配列）
b = tf.Variable(tf.zeros([1]))

#-- プレースホルダー --#
# 入力データを保持するプレースホルダー　形状(行数未指定, 2列)
x = tf.placeholder(tf.float32, shape=[None, 2])
# 正解データを保持するプレースホルダー　形状(行数未指定, 1列)
t = tf.placeholder(tf.float32, shape=[None, 1])

# シグモイド関数の出力を行うPOノード
out = tf.nn.sigmoid(tf.matmul(x, w) + b)
# 誤算関数（交差エントロピー誤差）のopノード
cost = - tf.reduce_sum(
    t * tf.log(out) + (1 - t) * tf.log(1 - out))
# 勾配降下アルゴリズムのopノード
train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# テストデータの予測を判定するopノード
correct_prediction = tf.equal(tf.to_float(tf.greater(out, 0.5)), t)
# 変数を初期化するopノード
init_op= tf.global_variables_initializer()

In [5]:
'''
セッション
'''
with tf.Session() as sess:
    sess.run(init_op)           # init_opノードを実行して変数を初期化
    
    # training_epochsの回数だけ学習を繰り返す
    for epoch in range(training_epochs):
        err, _ = sess.run(
            [cost,              # 交差エントロピー誤差関数ノードを実行
             train_op],         # ミニマイザーを実行
             feed_dict={x: X,   # 訓練データ
                        t: T    # 正解ラベル
                        }
            )
        epoch += 1              # epochに1を加算
        if epoch % 10 == 0:     # 10回ごとに誤差を出力
            print('(',epoch, ')',err)

    # 学習終了時の誤差を出力
    print('(',epoch, ')',err)
    
    # 学習結果で予測判定する
    print('\nCheck:')
    print(
        correct_prediction.eval(
            feed_dict={x: X,     # 訓練データをセット
                       t: T      # 正解ラベルをセット
                      }
        ))
    
    # 学習後のバイアスと重みを出力
    print(
        '\nb =\n', sess.run(b),
        '\nw1, w2 =\n', sess.run(w), '\n'
        )
    
    # 最終出力を表示
    print('Output Probability:') 
    print(
        out.eval(                # out2ノード(活性化関数)を実行
            feed_dict={x: X}
        ))

( 10 ) 2.1052206
( 20 ) 1.8430648
( 30 ) 1.657056
( 40 ) 1.509059
( 50 ) 1.3880336
( 60 ) 1.2870069
( 70 ) 1.2011769
( 80 ) 1.1271577
( 90 ) 1.0625086
( 100 ) 1.0054314
( 100 ) 1.0054314

Check:
[[ True]
 [ True]
 [ True]
 [ True]]

b =
 [-3.0316956] 
w1, w2 =
 [[1.7978806]
 [1.8669872]] 

Output Probability:
[[0.04601434]
 [0.2378128 ]
 [0.22551443]
 [0.6532084 ]]
